In [5]:

from bs4 import BeautifulSoup

soup = BeautifulSoup(open("mal1.html"), "html.parser")
sections = soup.find_all("section")
section = sections[0]
dls = section.find_all("dl")

section_name = section.find("h3").text.replace("\n", "")
section_dict = {}
for i in range(len(dls)):
#     print(i)
    dl = dls[i]
    key_ = dl.find("dt").text
    value_ = dl.find("dd").text.rsplit()
    section_dict[key_] = value_

print(section_dict["Tags:"])

['macros', 'macros-on-open', 'emotet-doc', 'emotet', 'generated-doc', 'opendir', 'loader', 'trojan']


In [6]:
section_dict["Tags:"]

['macros',
 'macros-on-open',
 'emotet-doc',
 'emotet',
 'generated-doc',
 'opendir',
 'loader',
 'trojan']

In [34]:
categories = {'ransomware', 'adware', 'spyware', 'worm', 'rootkit', 'trojan', 'backdoor'}
tag_dict = {"family": "", "category": ""}

In [36]:
matches = categories&set(section_dict["Tags:"])
length = len(matches)
print(length)

1


In [38]:
set(section_dict["Tags:"])-categories

{'emotet',
 'emotet-doc',
 'generated-doc',
 'loader',
 'macros',
 'macros-on-open',
 'opendir'}

In [14]:
l1 = [1,2,3,4]
l2 = ["a", "b", "c", "d"]

In [19]:
 
# list of strings 
lst = ['Geeks', 'For', 'Geeks', 'is', 'portal', 'for', 'Geeks'] 
  
# list of int 
lst2 = [11, 22, 33, 44, 55, 66, 77] 
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(l1, l2)), 
               columns =['Name', 'val']) 
df 

,Name,val
0,1,a
1,2,b
2,3,c
3,4,d


In [33]:

list(matches)[0]

'trojan'

In [42]:
categories = {'ransomware', 'adware', 'spyware', 'worm', 'rootkit', 'trojan', 'backdoor'}
tag_dict = {"family": "", "category": ""}

matches = categories&set(section_dict["Tags:"])
length = len(matches)

if length == 0:
    tag_dict["family"] = {"unknown"}
    tag_dict["category"] = {"unknown"}
elif length == 1:
    tag_dict["family"] = {"unknown"}
    tag_dict["category"] = list(matches)
elif length == 2:
    tag_dict["family"] = {"unknown"}
    tag_dict["category"] = list(matches)
else:
    tag_dict["family"] = set(section_dict["Tags:"])-categories
    tag_dict["category"] = list(matches) 

In [44]:
for tag in section_dict["Tags:"]:
    print("mal1.html       1       TAG    "+tag)

for family in tag_dict["family"] :
    print("mal1.html       1       TAG    "+family)
    
for cat in tag_dict["category"] :
    print("mal1.html       1       TAG    "+cat)

mal1.html       1       TAG    unknown
mal1.html       1       TAG    trojan


In [13]:


for tag in section_dict["Tags:"]:
#     print(tag)
    if tag in categories:
        print(tag)

trojan


In [9]:
for tag in section_dict["Tags:"]:
    print("mal1.html       1       TAG    "+tag)

mal1.html       1       TAG    macros
mal1.html       1       TAG    macros-on-open
mal1.html       1       TAG    emotet-doc
mal1.html       1       TAG    emotet
mal1.html       1       TAG    generated-doc
mal1.html       1       TAG    opendir
mal1.html       1       TAG    loader
mal1.html       1       TAG    trojan


In [1]:
import os
import sys
import csv
import json

OUTPUT_FORMATS = ('csv', 'json', 'yara', 'autofocus')

def getHandler(output_format):
    output_format = output_format.lower()
    if output_format not in OUTPUT_FORMATS:
        print("[WARNING] Invalid output format specified.. using CSV")
        output_format = 'csv'

    handler_format = "OutputHandler_" + output_format
    handler_class = getattr(sys.modules[__name__], handler_format)

    return handler_class()

class OutputHandler(object):
    def print_match(self, fpath, page, name, match, last = False):
        pass

    def print_header(self, fpath):
        pass

    def print_footer(self, fpath):
        pass

    def print_error(self, fpath, exception):
        print("[ERROR] %s" % (exception))

class OutputHandler_csv(OutputHandler):
    def __init__(self):
        self.csv_writer = csv.writer(sys.stdout, delimiter = '\t')

    def print_match(self, fpath, page, name, match):
        self.csv_writer.writerow((fpath, page, name, match))

    def print_error(self, fpath, exception):
        self.csv_writer.writerow((fpath, '0', 'error', exception))

class OutputHandler_json(OutputHandler):
    def print_match(self, fpath, page, name, match):
        data = {
            'path' : fpath,
            'file' : os.path.basename(fpath),
            'page' : page,
            'type' : name,
            'match': match
        }

        print(json.dumps(data))

    def print_error(self, fpath, exception):
        data = {
            'path'      : fpath,
            'file'      : os.path.basename(fpath),
            'type'      : 'error',
            'exception' : exception
        }

        print(json.dumps(data))

class OutputHandler_yara(OutputHandler):
    def __init__(self):
        self.rule_enc = ''.join(chr(c) if chr(c).isupper() or chr(c).islower() or chr(c).isdigit() else '_' for c in range(256))

    def print_match(self, fpath, page, name, match):
        if name in self.cnt:
            self.cnt[name] += 1
        else:
            self.cnt[name] = 1
        
        string_id = "$%s%d" % (name, self.cnt[name])
        self.sids.append(string_id)
        string_value = match.replace('\\', '\\\\')
        # print("\t\t%s = \"%s\"" % (string_id, string_value))

    def print_header(self, fpath):
        rule_name = os.path.splitext(os.path.basename(fpath))[0].translate(self.rule_enc)
        #
        # print("rule %s" % (rule_name))
        # print("{")
        # print("\tstrings:")
        #
        # self.cnt = {}
        # self.sids = []

    def print_footer(self, fpath):
        cond = ' or '.join(self.sids)
        #
        # print("\tcondition:")
        # print("\t\t" + cond)
        # print("}")

class OutputHandler_autofocus(OutputHandler):
    def __init__(self):
        self.rule_enc = ''.join(chr(c) if chr(c).isupper() or chr(c).islower() or chr(c).isdigit() else '_' for c in range(256))

    def print_match(self, fpath, page, name, match):
        string_value = match.replace('hxxp', 'http').replace('\\', '\\\\')

        # if name == "MD5":
        #     auto_focus_query = '{"field":"sample.md5","operator":"is","value":\"%s\"},' % (string_value)
        # elif name == "SHA1":
        #     auto_focus_query = '{"field":"sample.sha1","operator":"is","value":\"%s\"},' % (string_value)
        # elif name == "SHA256":
        #     auto_focus_query = '{"field":"sample.sha256","operator":"is","value":\"%s\"},' % (string_value)
        # elif name == "Tag":
        #     auto_focus_query = '{"field":"sample.sha256","operator":"is","value":\"%s\"},' % (string_value)
        # elif name == "URL":
        #     auto_focus_query = '{"field":"sample.tasks.connection","operator":"contains","value":\"%s\"},' % (string_value)
        # elif name == "Host":
        #     auto_focus_query = '{"field":"sample.tasks.dns","operator":"contains","value":\"%s\"},' % (string_value)
        # elif name == "Registry":
        #     #auto_focus_query = '{"field":"sample.tasks.registry","operator":"is","value":\"%s\"},' % (string_value)
        #     return
        # elif name == "Filepath":
        #     #auto_focus_query = '{"field":"sample.tasks.file","operator":"is","value":\"%s\"},' % (string_value)
        #     return
        # elif name == "Filename":
        #     #auto_focus_query = '{"field":"alias.filename","operator":"is","value":\"%s\"},' % (string_value)
        #     return
        # elif name == "Email":
        #     #auto_focus_query = '{"field":"alias.email","operator":"is","value":\"%s\"},' % (string_value)
        #     return
        # elif name == "IP":
        #     auto_focus_query = '{"field":"sample.tasks.connection","operator":"contains","value":\"%s\"},' % (string_value)
        # elif name == "CVE":
        #     return
        # print(auto_focus_query)

    def print_header(self, fpath):
        rule_name = os.path.splitext(os.path.basename(fpath))[0].translate(self.rule_enc)
        #
        # print("AutoFocus Search for: %s" % (rule_name))
        # print('{"operator":"Any","children":[')


    def print_footer(self, fpath):
        rule_name = os.path.splitext(os.path.basename(fpath))[0].translate(self.rule_enc)
        # print('{"field":"sample.tag","operator":"is in the list","value":[\"%s\"]}]}' % (rule_name))





In [6]:
import os
import glob
import re

class WhiteList(dict):
    def __init__(self, basedir):
        searchdir = os.path.join(basedir, "whitelists/whitelist_*.ini")
        fpaths = glob.glob(searchdir)
        for fpath in fpaths:
            t = os.path.splitext(fpath)[0].split('_',1)[1]
            patterns = [line.strip() for line in open(fpath)]
            self[t]  = [re.compile(p) for p in patterns]

In [8]:
import os
import sys
import fnmatch
import argparse
import re
# from StringIO import StringIO
try:
    import configparser as ConfigParser
except ImportError:
    import ConfigParser


import pandas as pd
from bs4 import BeautifulSoup
import requests


# Import additional project source files
import output
# from whitelist import WhiteList

In [11]:
os.path.dirname(os.path.abspath('mal1.html'))

'/home/asiri/Desktop/malware/ioc-parser'

In [12]:
class IOC_Parser(object):
    patterns = {}

    def __init__(self, patterns_ini='/home/asiri/Desktop/malware/ioc-parser/patterns.ini',
                 input_format = 'html', output_format='csv', dedup=False, library='pypdf2', path='mal1.html'):
        basedir = os.path.dirname(os.path.abspath(path))
        self.load_patterns(patterns_ini)
        self.whitelist = WhiteList(basedir)
        self.handler = output.getHandler(output_format)
        self.dedup = dedup

        self.ext_filter = "*." + input_format
        parser_format = "parse_" + input_format
        try:
            self.parser_func = getattr(self, parser_format)
        except AttributeError:
            e = 'Selected parser format is not supported: %s' % (input_format)
            raise NotImplementedError(e)

        self.library = library
        if input_format == 'pdf':
            if library not in IMPORTS:
                e = 'Selected PDF parser library not found: %s' % (library)
                raise ImportError(e)
        elif input_format == 'html':
            if 'beautifulsoup' not in IMPORTS:
                e = 'HTML parser library not found: BeautifulSoup'
                raise ImportError(e)

    def load_patterns(self, fpath):
        config = ConfigParser.ConfigParser()
        with open(fpath) as f:
            config.readfp(f)

        for ind_type in config.sections():
            try:
                ind_pattern = config.get(ind_type, 'pattern')
            except:
                continue

            if ind_pattern:
                ind_regex = re.compile(ind_pattern)
                self.patterns[ind_type] = ind_regex

    def is_whitelisted(self, ind_match, ind_type):
        for w in self.whitelist[ind_type]:
            if w.findall(ind_match):
                return True

        return False

    def parse_page(self, fpath, data, page_num):
        for ind_type, ind_regex in self.patterns.items():
            matches = ind_regex.findall(data)

            for ind_match in matches:
                if isinstance(ind_match, tuple):
                    ind_match = ind_match[0]

                if self.is_whitelisted(ind_match, ind_type):
                    continue

                if self.dedup:
                    if (ind_type, ind_match) in self.dedup_store:
                        continue

                    self.dedup_store.add((ind_type, ind_match))

                self.handler.print_match(fpath, page_num, ind_type, ind_match)


    def parse_html(self, f, fpath):
        try:
            if self.dedup:
                self.dedup_store = set()
                
            data = f.read()
            soup = BeautifulSoup(data, "html.parser")
            html = soup.findAll(text=True)
            sections = soup.find_all("section")
            section = sections[0]
            dls = section.find_all("dl")

            section_name = section.find("h3").text.replace("\n", "")
            section_dict = {}
            for i in range(len(dls)):
                #     print(i)
                dl = dls[i]
                key_ = dl.find("dt").text
                value_ = dl.find("dd").text.rsplit()
                section_dict[key_] = value_

            categories = {'ransomware', 'adware', 'spyware', 'worm', 'rootkit', 'trojan', 'backdoor'}
            tag_dict = {"family": "", "category": ""}

            matches = categories & set(section_dict["Tags:"])
            no_matches = set(section_dict["Tags:"]) - categories

            match_length = len(matches)

            if match_length == 0:
                matches = {"unknown"}
            if len(no_matches) == 0:
                no_matches = {"unknown"}

            length = len(section_dict["Tags:"])

            tag_dict["family"] = list(no_matches)
            tag_dict["category"] = list(matches)

            # if length == 0:
            #     tag_dict["family"] = {"unknown"}
            #     tag_dict["category"] = {"unknown"}
            # elif length == 1:
            #     tag_dict["family"] = {"unknown"}
            #     tag_dict["category"] = list(matches)
            # elif length == 2:
            #     tag_dict["family"] = {"unknown"}
            #     tag_dict["category"] = list(matches)
            # else:
            #     tag_dict["family"] = set(section_dict["Tags:"]) - categories
            #     tag_dict["category"] = list(matches)
            for tag in section_dict["Tags:"]:
                print(fpath+"       1       TAG    " + tag)

            for family in tag_dict["family"]:
                print(fpath+"       1       Family    " + family)

            for cat in tag_dict["category"]:
                print(fpath+"       1       Category    " + cat)

            text = u''
            for elem in html:
                if elem.parent.name in ['style', 'script', '[document]', 'head', 'title']:
                    continue
                elif re.match('<!--.*-->', unicode(elem)):
                    continue
                else:
                    text += unicode(elem)

            # self.handler.print_header(fpath)
            self.parse_page(fpath, text, 1)
            # self.handler.print_footer(fpath)
        except (KeyboardInterrupt, SystemExit):
            raise
        except Exception as e:
            self.handler.print_error(fpath, e)

    def parse(self, path):
        try:
            if path.startswith('http://') or path.startswith('https://'):
                if 'requests' not in IMPORTS:
                    e = 'HTTP library not found: requests'
                    raise ImportError(e)
                headers = { 'User-Agent': 'Mozilla/5.0 Gecko Firefox' }
                r = requests.get(path, headers=headers)
                r.raise_for_status()
                f = StringIO(r.content)
                self.parser_func(f, path)
                return
            elif os.path.isfile(path):
                with open(path, 'rb') as f:
                    self.parser_func(f, path)
                return
            elif os.path.isdir(path):
                for walk_root, walk_dirs, walk_files in os.walk(path):
                    for walk_file in fnmatch.filter(walk_files, self.ext_filter):
                        fpath = os.path.join(walk_root, walk_file)
                        with open(fpath, 'rb') as f:
                            self.parser_func(f, fpath)
                return

            e = 'File path is not a file, directory or URL: %s' % (path)
            raise IOError(e)
        except (KeyboardInterrupt, SystemExit):
            raise
        except Exception as e:
            self.handler.print_error(path, e)


In [13]:
IOC_Parser()

<ipython-input-12-b640f723760d>:33: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  config.readfp(f)
